In [13]:
import nbformat as nbf
import sys
import os
from pathlib import Path

import nbformat
import time
from nbconvert.preprocessors import ExecutePreprocessor

from nbconvert.nbconvertapp import NbConvertApp
from ipylab import JupyterFrontEnd

In [14]:
print(1)

1


In [15]:
print(2)

2


In [24]:
def check_execution_order(notebook_path,
                          check_all_executed=False,
                          check_all_except_last_executed=True,
                          check_top_to_bottom=False,
                          check_in_order=True,
                          exclude_last_cell=False):
    ntbk = nbf.read(notebook_path, nbf.NO_CONVERT)

    # extract all code cells (disregard markdown, raw and others), then extract the execution order
    output_cells = [cell for cell in ntbk.cells if cell["cell_type"] == "code"]
    # remove empty cells
    non_empty_cells = [cell for cell in output_cells if cell["source"] != ""]
    execution_counts = [cell["execution_count"] for cell in non_empty_cells]

    def _all_none(item_list):
        return all([i is None for i in item_list])

    # return early if no cells were executed
    if _all_none(execution_counts):
        return True

    pass_check = [True]

    def _check_all_executed(execution_counts: list) -> bool:
        """Check all cells were executed.

        Parameters
        ----------
        execution_counts : list
            execution_counts

        Returns
        -------
        bool
        """
        return not None in execution_counts

    def _check_in_order(execution_counts: list) -> bool:
        """Check that execution counts that aren't None go from 1 to N.

        Parameters
        ----------
        execution_counts : list
            execution counts

        Returns
        -------
        bool
        """
        print(execution_counts)
        execution_counts = [x for x in execution_counts if x is not None]
        count_range = len(execution_counts)
        if exclude_last_cell:
            count_range = count_range - 1
        is_in_order = all([execution_counts[i] < execution_counts[i + 1] for i in range(count_range - 1)])
        return is_in_order

    if check_in_order:
        pass_check.append(_check_in_order(execution_counts))

    if check_all_executed:
        pass_check.append(_check_all_executed(execution_counts))
    
    if check_all_except_last_executed:
        pass_check.append(_check_all_executed(execution_counts[:-1]))

    if check_top_to_bottom:
        pass_check.append(
            _check_all_executed(execution_counts) and _check_in_order(execution_counts)
        )
    return all(pass_check)


def save_ipynb():
    app = JupyterFrontEnd()
    print("Saving", end="")
    # note: docmanager:save doesn't lock the python thread until saving is completed.
    # Sometimes, new changes aren't completely saved before checks are performed.
    # Waiting for 0.1 seconds seems to prevent that.
    app.commands.execute('docmanager:save')
    time.sleep(0.1)
    print("")


def reload_notebook():
    app = JupyterFrontEnd()
    app.commands.execute('docmanager:reload')


def wait_for_user(message):
    proceed = input(message + " Y/n")
    if proceed.lower() == "y" or proceed == "":
        return True
    else:
        return False


def clear_and_rerun_notebook(notebook_filename, force_rerun=False, timeout=600):
    if "nbconvert_call" in sys.argv:
        print("Finished rerun")
        return

    save_ipynb()
    time.sleep(1)

    is_in_order = check_execution_order(notebook_filename)

    if is_in_order and not force_rerun:
        print("Notebook was already executed in order.")
        return
    else:
        rerun_confirmed_bool = wait_for_user("Notebook was not in order, rerun notebook now?")
        if not rerun_confirmed_bool and not force_rerun:
            print("Aborting.")
            return

    print("Rerunning.")
    with open(notebook_filename) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=timeout, kernel_name='python3', extra_arguments=["nbconvert_call"])
    ep.preprocess(nb, )

    with open(notebook_filename, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    reload_notebook()


def convert_ipynb(filepath, formats=None):
    if formats is None:
        formats = ["html", "python"]
    app = NbConvertApp()
    app.initialize()
    output_root_directory = os.path.join(r"C:\Users\ronal\PycharmProjects\git_lfs_test_2", "results",
                                         os.path.basename(filepath.replace('.', '_')))
    for format in formats:
        app.export_format = format
        app.notebooks = [filepath]
        app.output_base = os.path.join(output_root_directory, os.path.basename(filepath.replace('.ipynb', '')))
        if not os.path.exists(output_root_directory):
            os.makedirs(output_root_directory)
        app.start()



def export_all_figures():
    import junix

    for file in os.listdir(git_repo.working_dir):
        if file.endswith("ipynb"):
            file_without_extension = Path(file).stem
            images = junix.export_images(filepath=os.path.join(git_repo.working_dir, file),
                                         output_dir=os.path.join(git_repo.working_dir, "results",
                                                                 file.replace(".", "_")),
                                         prefix=file_without_extension)
            convert_ipynb(filepath=os.path.join(git_repo.working_dir, file), formats=["html"])


In [27]:
clear_and_rerun_notebook("test_jupyter.ipynb")

Saving
[13, 14, 15, 24, None, 25, 26]


Notebook was not in order, rerun notebook now? Y/n n


Aborting.


In [28]:
save_ipynb()
time.sleep(1)
check_execution_order("test_jupyter.ipynb")

Saving
[13, 14, 15, 24, 27, None, 26]


False

In [29]:
clear_and_rerun_notebook("test_jupyter.ipynb")

Saving
[13, 14, 15, 24, 27, 28, None]
Notebook was already executed in order.
